In [1]:
from pyspark import sql
from pyspark.sql import functions as f, udf

In [2]:
# Download the data files from AWS 

!wget -q  https://s3.amazonaws.com/metcs777/flights.csv.bz2
    
!wget -q  https://s3.amazonaws.com/metcs777/airports.csv.bz2

In [3]:
sqlContext = sql.SparkSession.builder \
    .master("local") \
    .appName("Flight DF") \
    .getOrCreate()

flights = sqlContext.read.format('csv')\
    .options(header='true', inferSchema='true')\
    .load("flights.csv.bz2")

airport = sqlContext.read.format('csv')\
    .options(header='true', inferSchema='true')\
    .load("airports.csv.bz2")

In [4]:
flights.printSchema()
airport.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

# Questions 

## Q1 Find a list of Origin Airports

In [5]:
#Q1 Find a list of Origin Airports
flights.select("ORIGIN_AIRPORT").distinct().show()

+--------------+
|ORIGIN_AIRPORT|
+--------------+
|           BGM|
|           PSE|
|           INL|
|           DLG|
|         12888|
|           MSY|
|           PPG|
|         12003|
|         15041|
|           GEG|
|           SNA|
|           BUR|
|           GRB|
|           GTF|
|         14986|
|         13851|
|           IDA|
|         11150|
|         15412|
|           GRR|
+--------------+
only showing top 20 rows



## Q2 Find a list of (Origin, Destination) pairs


In [6]:
#Q2 Find a list of (Origin, Destination) pairs
flights.select("ORIGIN_AIRPORT", "DESTINATION_AIRPORT").distinct().show()

+--------------+-------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|
+--------------+-------------------+
|           BQN|                MCO|
|           PHL|                MCO|
|           MCI|                IAH|
|           SPI|                ORD|
|           SNA|                PHX|
|           LBB|                DEN|
|           ORD|                PDX|
|           EWR|                STT|
|           ATL|                GSP|
|           MCI|                MKE|
|           PBI|                DCA|
|           SMF|                BUR|
|           MDW|                MEM|
|           LAS|                LIT|
|           TPA|                ACY|
|           DSM|                EWR|
|           FSD|                ATL|
|           SJC|                LIH|
|           CLE|                SJU|
|         11298|              11057|
+--------------+-------------------+
only showing top 20 rows



## Q3 Find the Origin airport which had the largest departure delay in the month of January


In [7]:
#Q3 Find the Origin airport which had the largest departure delay in the month of January
flights.where(flights.MONTH == 1)\
    .orderBy("DEPARTURE_DELAY", ascending=False)\
    .limit(1)\
    .select("ORIGIN_AIRPORT")\
    .show()

+--------------+
|ORIGIN_AIRPORT|
+--------------+
|           BHM|
+--------------+



## Q4 Find out which carrier has the largest delay on Weekends. 


In [8]:
#Q4 Find out which carrier has the largest delay on Weekends. 
flights.filter("DAY_OF_WEEK = 6 OR DAY_OF_WEEK = 7" )\
    .orderBy("DEPARTURE_DELAY", ascending=False)\
    .limit(1)\
    .select("AIRLINE")\
    .show()

+-------+
|AIRLINE|
+-------+
|     AA|
+-------+



## Q5 Which airport has the most cancellation of flights?


In [9]:
#Q5 Which airport has the most cancellation of flights?
flights.filter("CANCELLED = 1")\
    .withColumn("COUNT", f.lit(1))\
    .groupBy("ORIGIN_AIRPORT")\
    .agg(f.sum("COUNT").alias("COUNT"))\
    .orderBy("COUNT", ascending=False)\
    .limit(1)\
    .select("ORIGIN_AIRPORT", "COUNT")\
    .show()

+--------------+-----+
|ORIGIN_AIRPORT|COUNT|
+--------------+-----+
|           ORD| 8548|
+--------------+-----+



## Q6 Find the percent of flights cancelled for each carrier.


In [10]:
#Q6 Find the percent of flights cancelled for each carrier.
flights.withColumn("TOTAL", f.lit(1))\
    .groupBy("AIRLINE")\
    .agg(f.sum("CANCELLED").alias("TOTAL_CANCELLED"), f.sum("TOTAL").alias("TOTAL"))\
    .withColumn("CANCEL_RATE", f.col("TOTAL_CANCELLED")/f.col("TOTAL")*100)\
    .show()

+-------+---------------+-------+-------------------+
|AIRLINE|TOTAL_CANCELLED|  TOTAL|        CANCEL_RATE|
+-------+---------------+-------+-------------------+
|     UA|           6573| 515723|  1.274521400053905|
|     NK|           2004| 117379| 1.7072900604026275|
|     AA|          10919| 725984| 1.5040276369727157|
|     EV|          15231| 571977| 2.6628693111785964|
|     B6|           4276| 267048| 1.6012102693148795|
|     DL|           3824| 875881| 0.4365889886868193|
|     OO|           9960| 588353| 1.6928612584621818|
|     F9|            588|  90836| 0.6473204456382932|
|     US|           4067| 198715| 2.0466497244797823|
|     MQ|          15025| 294632| 5.0995818512585185|
|     HA|            171|  76272|0.22419760855884205|
|     AS|            669| 172521|   0.38777887909298|
|     VX|            534|  61903| 0.8626399366751207|
|     WN|          16043|1261855| 1.2713822111098343|
+-------+---------------+-------+-------------------+



## Q7 Find the largest departure delay for each carrier


In [11]:
#Q7 Find the largest departure delay for each carrier
flights.groupBy("AIRLINE")\
    .agg(f.max("DEPARTURE_DELAY").alias("MAX_DEPARTURE_DELAY"))\
    .show()

+-------+-------------------+
|AIRLINE|MAX_DEPARTURE_DELAY|
+-------+-------------------+
|     UA|               1314|
|     NK|                836|
|     AA|               1988|
|     EV|               1274|
|     B6|               1006|
|     DL|               1289|
|     OO|               1378|
|     F9|               1112|
|     US|                759|
|     MQ|               1544|
|     HA|               1433|
|     AS|                963|
|     VX|                644|
|     WN|                665|
+-------+-------------------+



In [12]:
#Q8 Find the largest departure delay for each carrier for each month
flights.groupBy("AIRLINE", "MONTH")\
    .agg(f.max("DEPARTURE_DELAY").alias("MAX_DEPARTURE_DELAY"))\
    .show()

+-------+-----+-------------------+
|AIRLINE|MONTH|MAX_DEPARTURE_DELAY|
+-------+-----+-------------------+
|     NK|   11|                476|
|     VX|   10|                430|
|     UA|   12|               1194|
|     HA|   10|               1022|
|     OO|    3|                874|
|     OO|    4|                878|
|     OO|    9|                893|
|     F9|    2|                852|
|     F9|   12|                781|
|     HA|    5|                326|
|     UA|    4|               1314|
|     MQ|   10|               1544|
|     HA|   12|               1095|
|     EV|    4|                757|
|     DL|    6|               1201|
|     DL|    3|               1166|
|     DL|    8|               1207|
|     B6|    6|                507|
|     DL|   10|               1120|
|     OO|   10|               1122|
+-------+-----+-------------------+
only showing top 20 rows



## Q8 Find the largest departure delay for each carrier for each month
 

In [13]:
#Q9 For each carrier find the average Departure delay
flights.withColumn("TOTAL", f.lit(1))\
    .groupBy("AIRLINE")\
    .agg(f.sum("DEPARTURE_DELAY").alias("TOTAL_DEPARTURE_DELAY"), f.sum("TOTAL").alias("TOTAL"))\
    .withColumn("AVG_DEPARTURE_DELAY", f.col("TOTAL_DEPARTURE_DELAY")/f.col("TOTAL"))\
    .show()

+-------+---------------------+-------+-------------------+
|AIRLINE|TOTAL_DEPARTURE_DELAY|  TOTAL|AVG_DEPARTURE_DELAY|
+-------+---------------------+-------+-------------------+
|     UA|              7355348| 515723|  14.26220664969373|
|     NK|              1840887| 117379|  15.68327383944317|
|     AA|              6369435| 725984|  8.773519802089302|
|     EV|              4857338| 571977|   8.49219111957299|
|     B6|              3026467| 267048| 11.333044995656211|
|     DL|              6427294| 875881|  7.338090448359994|
|     OO|              4517510| 588353|   7.67823058605973|
|     F9|              1205449|  90836|  13.27060856928971|
|     US|              1196447| 198715| 6.0209194071912036|
|     MQ|              2837908| 294632|   9.63204268375465|
|     HA|                36972|  76272|0.48473882945248586|
|     AS|               306997| 172521| 1.7794761217474975|
|     VX|               553852|  61903|  8.947094648078446|
|     WN|             13186520|1261855| 

## Q10 For each carrier find the average Departure delay for each month


In [17]:
#Q10 For each carrier find the average Departure delay for each month
flights.withColumn("TOTAL", f.lit(1))\
    .groupBy("AIRLINE","MONTH")\
    .agg(f.sum("DEPARTURE_DELAY").alias("TOTAL_DEPARTURE_DELAY"), f.sum("TOTAL").alias("TOTAL"))\
    .withColumn("AVG_DEPARTURE_DELAY", f.col("TOTAL_DEPARTURE_DELAY")/f.col("TOTAL"))\
    .select("AIRLINE", "MONTH", "AVG_DEPARTURE_DELAY")\
    .show()

+-------+-----+-------------------+
|AIRLINE|MONTH|AVG_DEPARTURE_DELAY|
+-------+-----+-------------------+
|     NK|   11|  8.559720582447856|
|     VX|   10|  7.053440702781844|
|     UA|   12|  17.51819625716456|
|     HA|   10| 0.1680551105414931|
|     OO|    3|  5.789528335796158|
|     OO|    4|  5.276855399460763|
|     OO|    9| 3.8390551181102364|
|     F9|    2| 25.258564296780857|
|     F9|   12|  15.89396551724138|
|     HA|    5|-1.3484612993472178|
|     UA|    4| 12.880508925547868|
|     MQ|   10| 3.4174779364934946|
|     HA|   12|-0.4426517571884984|
|     EV|    4|  6.673949204803635|
|     DL|    6| 10.844916186654585|
|     DL|    3|  8.386646172100424|
|     DL|    8|   7.74069452851866|
|     B6|    6|  11.31625144073056|
|     DL|   10| 3.2151895383312157|
|     OO|   10| 3.6338714964759875|
+-------+-----+-------------------+
only showing top 20 rows



## Q11 Which date of year has the highest rate  of flight cancellations? Rate of flight cancellation is calculated by deviding number of canceled flights by total number of flights.


In [18]:
#Q11 Which date of year has the highest rate  of flight cancellations?
# Rate of flight cancellation is calculated by deviding number of canceled flights by total number of flights.

flights.withColumn("TOTAL", f.lit(1))\
    .groupBy("YEAR","MONTH","DAY")\
    .agg(f.sum("CANCELLED").alias("TOTAL_CANCELLED"), f.sum("TOTAL").alias("TOTAL"))\
    .withColumn("CANCEL_RATE", f.col("TOTAL_CANCELLED")/f.col("TOTAL")*100)\
    .orderBy("CANCEL_RATE", ascending=False)\
    .limit(1)\
    .select("YEAR","MONTH","DAY")\
    .show()


+----+-----+---+
|YEAR|MONTH|DAY|
+----+-----+---+
|2015|    1| 27|
+----+-----+---+



## Q12 Calculate the number of flights to each destination state For each carrier, for which state do they have the largest average delay? You will need the airline and airport data sets for this question.

In [19]:
#Q12 Calculate the number of flights to each destination state
# For each carrier, for which state do they have the largest average delay? 
# You will need the airline and airport data sets for this question.

from pyspark.sql.types import ArrayType, IntegerType, StringType
from pyspark.sql.functions import udf
fold_list = udf(lambda x,y: sorted(zip(x,y))[-1][1],StringType())

#Q8 Find the largest departure delay for each carrier for each month
flights.withColumn("COUNT", f.lit(1))\
    .groupBy("AIRLINE", "DESTINATION_AIRPORT")\
    .agg(f.sum("DEPARTURE_DELAY").alias("TOTAL_DEPARTURE_DELAY"),f.sum("COUNT").alias("COUNT"))\
    .withColumn("DEPARTURE_AVG_DELAY", f.col("TOTAL_DEPARTURE_DELAY")/f.col("COUNT"))\
    .join(airport, flights.DESTINATION_AIRPORT == airport.IATA_CODE)\
    .select("AIRLINE", "TOTAL_DEPARTURE_DELAY", "STATE")\
    .groupBy("AIRLINE")\
    .agg(
        f.collect_list("TOTAL_DEPARTURE_DELAY").alias("delay"),
        f.collect_list("STATE").alias("state")
    )\
    .withColumn("MAX_AVGDELAY_STATE", fold_list(f.col("delay"), f.col("state")))\
    .select("AIRLINE", "MAX_AVGDELAY_STATE")\
    .show()


+-------+------------------+
|AIRLINE|MAX_AVGDELAY_STATE|
+-------+------------------+
|     UA|                IL|
|     NK|                IL|
|     AA|                TX|
|     EV|                GA|
|     B6|                NY|
|     DL|                GA|
|     OO|                IL|
|     F9|                CO|
|     US|                NC|
|     MQ|                IL|
|     HA|                HI|
|     AS|                WA|
|     VX|                CA|
|     WN|                IL|
+-------+------------------+

